In [ ]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import db_utils as du
import create_tables as ct


conn = ct.create_or_open_database()
df_fahrrad_Zähler = du.fetch_data_df('Standorten_Zählstelle',conn)
df_mess_Fahrrad= du.fetch_data_df('Messdaten_Fahrrad',conn)
df_bezirke= du.fetch_data_df('Bezirke',conn)
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)


In [ ]:
# in order to make fetching data from the database faster:
# creation of index for messdaten_auto und messDaten_Fahrrad
# conn.isolation_level = None # deaktiviert Autocommit
#try to use chunksize in the fetch_data . but we have to define parameter 

In [ ]:
import pandas as pd
df_merged = pd.merge(df_fahrrad_Zähler, df_bezirke, on="Bezirk", how="inner")

df_merged = pd.merge(df_merged, df_mess_Fahrrad, on="Zählstelle", how="inner")

# Merge with Date_dim on Installationsdatum and Date
df_merged = pd.merge(df_merged, df_date, on="DateID", how="inner")

# Merge with time_dim on Installationsdatum time
df_merged = pd.merge(df_merged, df_time, on='TimeID', how="inner")

# Display the merged da
df_merged.head()

In [ ]:
df_merged.columns

In [5]:
df_merged.drop(columns=['Installationsdatum', 'Breitengrad', 'Längengrad', 'Geometry'],axis=1,  inplace= True)


In [ ]:
df_merged.head()

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

plt.figure(figsize=(10, 6))
#X :Month Y: Wert Hue: Bezirke
sns.scatterplot( x= df_merged['month'],y=df_merged['Wert'], hue=df_merged['Bezirk'])
plt.show()


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Function to process and visualize data chunks
def process_chunk(chunk):
    # Here you can perform any data manipulations or visualizations
    # Example: Calculate mean for 'q_pkw_mq_hr' grouped by 'DateID' and 'stunde'
    aggregated_data = chunk.groupby(['DateID', 'stunde'])['q_pkw_mq_hr'].mean().reset_index()

    # Example Visualization: Plotting the mean 'q_pkw_mq_hr' over time
    plt.figure(figsize=(10, 6))
    plt.plot(aggregated_data['stunde'], aggregated_data['q_pkw_mq_hr'], label="Average q_pkw_mq_hr")
    plt.xlabel('Hour of the Day')
    plt.ylabel('Average q_pkw_mq_hr')
    plt.title('Average q_pkw_mq_hr by Hour of the Day')
    plt.legend()
    plt.show()


In [ ]:
chunks = du.fetch_data_df_chunk(conn)

# Step 4: Process and visualize each chunk
for chunk in chunks:
    process_chunk(chunk)